In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [2]:
start = dt.datetime(2010,1,1)
end = dt.datetime(2022,12,31)

df = yf.download('AAPL',start,end)
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.505279,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.516528,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.412873,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.401017,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.443572,447610800


In [3]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.658981,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,129.831772,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,125.847855,85438400
2022-12-29,127.989998,130.479996,127.730003,129.610001,129.412415,75703700
2022-12-30,128.410004,129.949997,127.430000,129.929993,129.731918,76960600


In [4]:
df = df.reset_index()
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.505279,493729600
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.516528,601904800
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.412873,552160000
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.401017,477131200
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.443572,447610800


In [5]:
df = df.drop(['Date','Adj Close'], axis = 1)
df.head()

,Open,High,Low,Close,Volume
0,7.622500,7.660714,7.585000,7.643214,493729600
1,7.664286,7.699643,7.616071,7.656429,601904800
2,7.656429,7.686786,7.526786,7.534643,552160000
3,7.562500,7.571429,7.466071,7.520714,477131200
4,7.510714,7.571429,7.466429,7.570714,447610800


In [6]:
df.shape

(3272, 5)

In [30]:
df.tail()

,Open,High,Low,Close,Volume
3267,130.919998,132.419998,129.639999,131.860001,63814900
3268,131.380005,131.410004,128.720001,130.029999,69007800
3269,129.669998,131.029999,125.870003,126.040001,85438400
3270,127.989998,130.479996,127.730003,129.610001,75703700
3271,128.410004,129.949997,127.430000,129.929993,76960600


In [8]:
# Splitting the data into training and testing set

data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70): int(len(df))])

print(data_training.shape)
print(data_testing.shape)# Splitting the data into training and testing set

(2290, 1)
(982, 1)


In [9]:
data_training

,Close
0,7.643214
1,7.656429
2,7.534643
3,7.520714
4,7.570714
...,...
2285,41.630001
2286,42.812500
2287,43.544998
2288,43.560001


In [10]:
scaler = MinMaxScaler(feature_range=(0,1))

In [11]:
data_training_array = scaler.fit_transform(data_training)

[[0.01533047]
 [0.01558878]
 [0.01320823]
 ...
 [0.71710501]
 [0.71739828]
 [0.70127194]]


In [12]:
data_training_array.shape

(2290, 1)

In [23]:
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100: i])
    y_train.append(data_training_array[i,0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
print(x_train.shape)
print(y_train.shape)

(2190, 100, 1)
(2190,)


In [25]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (x_train.shape[1], 1) ))
model.add(Dropout(0.2))

model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(units = 1))

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 100, 50)           10400     
                                                                 
 dropout_4 (Dropout)         (None, 100, 50)           0         
                                                                 
 lstm_5 (LSTM)               (None, 100, 60)           26640     
                                                                 
 dropout_5 (Dropout)         (None, 100, 60)           0         
                                                                 
 lstm_6 (LSTM)               (None, 100, 80)           45120     
                                                                 
 dropout_6 (Dropout)         (None, 100, 80)           0         
                                                                 
 lstm_7 (LSTM)               (None, 120)              

In [27]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 10)

Epoch 1/10
69/69 [==============================] - 25s 254ms/step - loss: 0.0228
Epoch 2/10
69/69 [==============================] - 18s 258ms/step - loss: 0.0058
Epoch 3/10
69/69 [==============================] - 17s 248ms/step - loss: 0.0048
Epoch 4/10
69/69 [==============================] - 15s 222ms/step - loss: 0.0051
Epoch 5/10
69/69 [==============================] - 17s 239ms/step - loss: 0.0050
Epoch 6/10
69/69 [==============================] - 17s 247ms/step - loss: 0.0040
Epoch 7/10
69/69 [==============================] - 17s 243ms/step - loss: 0.0040
Epoch 8/10
69/69 [==============================] - 17s 244ms/step - loss: 0.0041
Epoch 9/10
69/69 [==============================] - 17s 244ms/step - loss: 0.0036
Epoch 10/10
69/69 [==============================] - 17s 251ms/step - loss: 0.0040


In [28]:
data_testing.head()

,Close
2290,42.602501
2291,42.357498
2292,42.722500
2293,42.544998
2294,42.700001


In [29]:
data_testing.tail()

,Close
3267,131.860001
3268,130.029999
3269,126.040001
3270,129.610001
3271,129.929993


In [31]:
last_100_days = data_testing.tail(100)
last_100_days

,Close
3172,169.240005
3173,168.490005
3174,172.100006
3175,173.190002
3176,173.029999
...,...
3267,131.860001
3268,130.029999
3269,126.040001
3270,129.610001


In [32]:
final_df = last_100_days

In [33]:
final_df

,Close
3172,169.240005
3173,168.490005
3174,172.100006
3175,173.190002
3176,173.029999
...,...
3267,131.860001
3268,130.029999
3269,126.040001
3270,129.610001


In [34]:
input_data = scaler.fit_transform(final_df)
input_data

array([[0.89053809],
       [0.87507736],
       [0.94949501],
       [0.97196453],
       [0.96866617],
       [1.        ],
       [0.99175409],
       [0.93753868],
       [0.85611224],
       [0.84910313],
       [0.85528749],
       [0.90682325],
       [0.77468548],
       [0.72850964],
       [0.67759228],
       [0.64275405],
       [0.65800875],
       [0.6136878 ],
       [0.58730152],
       [0.61678014],
       [0.58585868],
       [0.64584607],
       [0.77076871],
       [0.5730776 ],
       [0.60338065],
       [0.54277454],
       [0.50834869],
       [0.58627074],
       [0.63615732],
       [0.57060398],
       [0.55040205],
       [0.50278274],
       [0.50979184],
       [0.5301998 ],
       [0.49062037],
       [0.33889907],
       [0.25066987],
       [0.33828067],
       [0.41352307],
       [0.41970711],
       [0.39971121],
       [0.2896309 ],
       [0.29643366],
       [0.26674901],
       [0.25355586],
       [0.34941257],
       [0.25438061],
       [0.337

In [35]:
input_data.shape

(100, 1)

In [145]:
x_test = []
for i in range(100, input_data.shape[0]+1):
    x_test.append(input_data[i-100:i])
    
x_test = np.array(x_test)

In [146]:
for i in range (0,7):
    y_predicted = model.predict(x_test)
    temp = np.asarray(y_predicted[-1]).reshape(1,1)
    x_temp = x_test[len(x_test)-1][1:100]
    x_temp=np.append(x_temp,temp,axis=0)
    x_temp = np.asarray(x_temp).reshape(1,100,1)
    x_test = np.append(x_test,x_temp,axis=0)

1/1 [==============================] - 0s 87ms/step


In [147]:
y_predicted

array([[0.32573646],
       [0.30984992],
       [0.29475147],
       [0.28103924],
       [0.2691821 ],
       [0.25952286],
       [0.2522108 ]], dtype=float32)